In [ ]:
#1.remove_double_quotes_from_value_of_json_string.py
"""
example: {"id":"1","name":"neeraj"kumar","city":"del"} 
i.e. to remove " in "neeraj kumar"

Split function in python:
s1="a|b|c"
print(s1.split('|'))
"""

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split,col,lit,concat_ws,concat

spark=SparkSession.builder.getOrCreate()

In [ ]:
jsonString="""{"id":"1","name":"neeraj"kumar","city":"del"}"""
print(jsonString)


{"id":"1","name":"neeraj"kumar","city":"del"}


In [ ]:

data=[(1,'{"id":"1","name":"nitin","city":"blr"}'),(2,jsonString)]
cols=["col1","col2"]

df=spark.createDataFrame(data,cols)
display(df)

col1,col2
1,"{""id"":""1"",""name"":""nitin"",""city"":""blr""}"
2,"{""id"":""1"",""name"":""neeraj""kumar"",""city"":""del""}"


In [ ]:
split_df = df.withColumn("col3",split(col('col2'),'"name":"')[0]) \
.withColumn("col4",lit('"name":"')) \
.withColumn("col5",split(col('col2'),'"name":"')[1])

display(split_df)

split_two_parts_df = split_df.withColumn("col6",split(col("col5"),'"',2))

display(split_two_parts_df)

#Spark SQL functions lit() used to add a new column to DataFrame by assigning a literal or constant value. This function return Column type as return type. 

col1,col2,col3,col4,col5
1,"{""id"":""1"",""name"":""nitin"",""city"":""blr""}","{""id"":""1"",","""name"":""","nitin"",""city"":""blr""}"
2,"{""id"":""1"",""name"":""neeraj""kumar"",""city"":""del""}","{""id"":""1"",","""name"":""","neeraj""kumar"",""city"":""del""}"


col1,col2,col3,col4,col5,col6
1,"{""id"":""1"",""name"":""nitin"",""city"":""blr""}","{""id"":""1"",","""name"":""","nitin"",""city"":""blr""}","List(nitin, ,""city"":""blr""})"
2,"{""id"":""1"",""name"":""neeraj""kumar"",""city"":""del""}","{""id"":""1"",","""name"":""","neeraj""kumar"",""city"":""del""}","List(neeraj, kumar"",""city"":""del""})"


In [ ]:
split_concat_df = split_two_parts_df.withColumn("col7",concat_ws(' ',col("col6")))
display(split_concat_df)

col1,col2,col3,col4,col5,col6,col7
1,"{""id"":""1"",""name"":""nitin"",""city"":""blr""}","{""id"":""1"",","""name"":""","nitin"",""city"":""blr""}","List(nitin, ,""city"":""blr""})","nitin ,""city"":""blr""}"
2,"{""id"":""1"",""name"":""neeraj""kumar"",""city"":""del""}","{""id"":""1"",","""name"":""","neeraj""kumar"",""city"":""del""}","List(neeraj, kumar"",""city"":""del""})","neeraj kumar"",""city"":""del""}"


In [ ]:
merged_df = split_concat_df.withColumn("col8",concat(col("col3"),col("col4"),col("col7")))
display(merged_df)

col1,col2,col3,col4,col5,col6,col7,col8
1,"{""id"":""1"",""name"":""nitin"",""city"":""blr""}","{""id"":""1"",","""name"":""","nitin"",""city"":""blr""}","List(nitin, ,""city"":""blr""})","nitin ,""city"":""blr""}","{""id"":""1"",""name"":""nitin ,""city"":""blr""}"
2,"{""id"":""1"",""name"":""neeraj""kumar"",""city"":""del""}","{""id"":""1"",","""name"":""","neeraj""kumar"",""city"":""del""}","List(neeraj, kumar"",""city"":""del""})","neeraj kumar"",""city"":""del""}","{""id"":""1"",""name"":""neeraj kumar"",""city"":""del""}"


In [ ]:
final_df=merged_df.select(col("col2"),col("col8"))
display(final_df)

final_df.show(truncate=False)

final_df.collect()

col2,col8
"{""id"":""1"",""name"":""nitin"",""city"":""blr""}","{""id"":""1"",""name"":""nitin ,""city"":""blr""}"
"{""id"":""1"",""name"":""neeraj""kumar"",""city"":""del""}","{""id"":""1"",""name"":""neeraj kumar"",""city"":""del""}"


+---------------------------------------------+---------------------------------------------+
|col2                                         |col8                                         |
+---------------------------------------------+---------------------------------------------+
|{"id":"1","name":"nitin","city":"blr"}       |{"id":"1","name":"nitin ,"city":"blr"}       |
|{"id":"1","name":"neeraj"kumar","city":"del"}|{"id":"1","name":"neeraj kumar","city":"del"}|
+---------------------------------------------+---------------------------------------------+

Out[46]: [Row(col2='{"id":"1","name":"nitin","city":"blr"}', col8='{"id":"1","name":"nitin ,"city":"blr"}'),
 Row(col2='{"id":"1","name":"neeraj"kumar","city":"del"}', col8='{"id":"1","name":"neeraj kumar","city":"del"}')]